<br>

# Técnicas Matemáticas para Big Data - Project NN?
<br><br>


GROUP NN:
- Rodrigo Pinto - Nº 103280 - ??% Work Participation
- Inês Silva - Nº 134870 - ??% Work Participation

<br><br>

## 1. Introduction to the problem of study [1,0 valor]

GitHub is a collaborative development platform that promotes collaboration between developers. This platform allows multiple people to work simultaneously on the same project, using commands such as pull requests, commits, and others. Analyzing these activities allows us to understand collaboration patterns, measure developer activity, and more.

One of the main challenges in this analysis is determining the number of unique users interacting with repositories in a given period of time, such as each month. Accurately counting these users requires the storage and processing of huge volumes of data, which leads to high memory costs.

A possible solution to this problem is the use of the HyperLogLog (HLL) algorithm. This algorithm allows for the efficient estimation of the number of unique elements in large datasets.

In this project, the objective is to apply the Hyperloglog algorithm to estimate the number of unique users interacting daily with a GitHub repository.

<br><br>
## 2. Brief and general description of the approach and methods used [1,5 valor]

<br><br>
## 3. Brief History and literature review of the problem and methods/algorithms [1,5 valor]

<br><br>
## 4. About the main method/algorithm used [1,5 valor]

<br><br>

## 5. Python imports and global configurations [0,5 valor]

### Install and import the necessary libraries to compute the Bayesian Network and perform other methods  

In [ ]:
# %pip install pandas
# %pip install seaborn
# %pip install matplotlib
# %pip install numpy
# %pip install pomegranate
# %pip install torch
# %pip install Pillow
%pip install duckdb pandas requests tqdm
%pip install datasketch
%pip install pybloom-live
%pip install hyperloglog
import time
import math, hashlib
import duckdb
import pandas as pd
from pybloom_live import BloomFilter
import os
import gzip
import json
import requests
from datetime import datetime, timedelta
from tqdm import tqdm
import hyperloglog
import redis
import random
import sys



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


<br><br>

## 6. Dataset and variables explanation [1,5 valor]

### 6.1 Data Retrieval

In [ ]:
# ----------------------------------------------
# CONFIGURAÇÕES
# ----------------------------------------------
# Pasta local onde os ficheiros .json.gz serão guardados
DATA_DIR = "gharchive_data"
os.makedirs(DATA_DIR, exist_ok=True)

# Base de dados DuckDB
DB_PATH = "github_archive.duckdb"

# Intervalo de tempo (exemplo: 3 dias)
START_DATE = datetime(2024, 10, 1)
END_DATE = datetime(2024, 10, 2)  # até, mas não incluindo este dia

# ----------------------------------------------
# FUNÇÃO PARA DOWNLOAD E LEITURA DE UM FICHEIRO
# ----------------------------------------------
def download_and_parse(date_obj, hour):
    """Faz download de um ficheiro GHArchive para uma data/hora específica."""
    date_str = date_obj.strftime("%Y-%m-%d")
    url = f"https://data.gharchive.org/{date_str}-{hour}.json.gz"
    local_path = os.path.join(DATA_DIR, f"{date_str}-{hour}.json.gz")

    # Se já existir localmente, não faz download novamente
    if not os.path.exists(local_path):
        r = requests.get(url, timeout=60)
        if r.status_code == 200:
            with open(local_path, "wb") as f:
                f.write(r.content)
        else:
            print(f"[Aviso] Falhou o download de {url} (status {r.status_code})")
            return []

    # Lê o conteúdo
    with gzip.open(local_path, "rb") as f:
        data = f.read().decode("utf-8").splitlines()

    # Extrai apenas os campos relevantes
    records = []
    for line in data:
        try:
            e = json.loads(line)
            records.append({
                "actor_id": e.get("actor", {}).get("id"),
                "actor_login": e.get("actor", {}).get("login"),
                "repo_name": e.get("repo", {}).get("name"),
                "type": e.get("type"),
                "created_at": e.get("created_at")
            })
        except json.JSONDecodeError:
            continue

    return records

# ----------------------------------------------
# LOOP PRINCIPAL DE DOWNLOAD + ARMAZENAMENTO
# ----------------------------------------------
all_records = []

current_date = START_DATE
while current_date < END_DATE:
    for hour in tqdm(range(24), desc=f"Processando {current_date.date()}"):
        records = download_and_parse(current_date, hour)
        all_records.extend(records)
    current_date += timedelta(days=1)

# Converte para DataFrame
df = pd.DataFrame(all_records)
print(f"\nTotal de eventos recolhidos: {len(df):,}")

# ----------------------------------------------
# GRAVAÇÃO NA BASE DE DADOS DUCKDB
# ----------------------------------------------
con = duckdb.connect(DB_PATH)
con.execute("""
CREATE TABLE IF NOT EXISTS github_events (
    actor_id BIGINT,
    actor_login VARCHAR,
    repo_name VARCHAR,
    type VARCHAR,
    created_at TIMESTAMP
)
""")

con.register("tmp_df", df)
con.execute("INSERT INTO github_events SELECT * FROM tmp_df")
con.close()

print(f"\n✅ Dados guardados com sucesso em {DB_PATH}")


Processando 2024-10-01:  29%|██▉       | 7/24 [01:19<03:11, 11.29s/it]


KeyboardInterrupt: 

### 6.2 Data Reading

In [5]:
# Caminhos
DATA_DIR = "gharchive_data"
DB_PATH = "github_archive.duckdb"

# Função para ler ficheiro local
def read_local_file(path):
    """Lê ficheiro JSON.GZ local e devolve lista de registos relevantes."""
    with gzip.open(path, "rb") as f:
        data = f.read().decode("utf-8").splitlines()

    records = []
    for line in data:
        try:
            e = json.loads(line)
            records.append({
                "actor_id": e.get("actor", {}).get("id"),
                "actor_login": e.get("actor", {}).get("login"),
                "repo_name": e.get("repo", {}).get("name"),
                "type": e.get("type"),
                "created_at": e.get("created_at")
            })
        except json.JSONDecodeError:
            continue
    return records


# Lê todos os ficheiros .json.gz da pasta
files = sorted([f for f in os.listdir(DATA_DIR) if f.endswith(".json.gz")])

all_records = []
for file in tqdm(files, desc="A carregar ficheiros locais"):
    path = os.path.join(DATA_DIR, file)
    records = read_local_file(path)
    all_records.extend(records)

# Converte para DataFrame
df = pd.DataFrame(all_records)
print(f"\nTotal de eventos extraídos: {len(df):,}")

# Conexão à base DuckDB
con = duckdb.connect(DB_PATH)

# Cria tabela se não existir
con.execute("""
CREATE TABLE IF NOT EXISTS github_events (
    actor_id BIGINT,
    actor_login VARCHAR,
    repo_name VARCHAR,
    type VARCHAR,
    created_at TIMESTAMP
)
""")

# Regista dataframe temporário e insere na tabela
con.register("tmp_df", df)
con.execute("INSERT INTO github_events SELECT * FROM tmp_df")
con.close()

print(f"✅ Inseridos {len(df):,} novos registos na base {DB_PATH}")

A carregar ficheiros locais: 100%|██████████| 48/48 [10:45<00:00, 13.44s/it]



Total de eventos extraídos: 10,174,100
✅ Inseridos 10,174,100 novos registos na base github_archive.duckdb


### 6.3 Dataset Analysis

In [48]:
# Ligação à base existente
con = duckdb.connect("github_archive.duckdb")

# Leitura dos dados necessários
df = con.execute("""
    SELECT actor_id, actor_login, repo_name, type, created_at
    FROM github_events
    WHERE actor_login IS NOT NULL
""").fetchdf()

print(f"------- Small view of the dataset: -------\n{df.head()}")
print(f"\n------- Datatypes: ------- \n{df.dtypes}")
print(f"\n------- Missing values: -------\n{df.isnull().sum()}")

print(f"\nTotal events read: {len(df):,}")
print(f"\nTotal users (w/ repetion): {df['actor_login'].nunique():,}")

con.close()


------- Small view of the dataset: -------
   actor_id          actor_login                    repo_name       type  \
0  74199667              Chroiud          Chroiud/playercount  PushEvent   
1   5622661          utilForever              utilForever/BOJ  PushEvent   
2  49699333      dependabot[bot]  mppanayotov/angular-capital  PushEvent   
3  41898282  github-actions[bot]            ePlus-DEV/metrics  PushEvent   
4  49556955        JonatasAlves9         JonatasAlves9/Doctor  PushEvent   

  created_at  
0 2024-10-01  
1 2024-10-01  
2 2024-10-01  
3 2024-10-01  
4 2024-10-01  

------- Datatypes: ------- 
actor_id                int64
actor_login            object
repo_name              object
type                   object
created_at     datetime64[us]
dtype: object

------- Missing values: -------
actor_id       0
actor_login    0
repo_name      0
type           0
created_at     0
dtype: int64

Total events read: 10,174,100

Total users (w/ repetion): 1,123,357


<br><br>

## 7. Main code as possible solution to the problem [1,5 valor] 

### 7.1 HLL "by hand"

In [49]:
# -----------------------------------
# FUNÇÕES HLL
# -----------------------------------

def hash_function(value):
    hash_value = hashlib.sha256(str(value).encode('utf8')).hexdigest()
    return int(hash_value, 16)

def leftmost_1_bit_position(hash_value):
    bin_hash = bin(hash_value)[2:]
    for i, bit in enumerate(bin_hash):
        if bit == '1':
            return i + 1
    return len(bin_hash) + 1

def process_element(element, registers, m):
    hash_value = hash_function(element)
    p = int(math.log2(m))
    register_index = hash_value & (m - 1)
    remaining_hash = hash_value >> p
    position = leftmost_1_bit_position(remaining_hash)
    registers[register_index] = max(registers[register_index], position)

def harmonic_mean(registers):
    sum_of_inverses = sum([2**-reg for reg in registers])
    return len(registers) / sum_of_inverses

def bias_correction(registers, raw_estimate, m):
    if raw_estimate <= 2.5 * m:
        V = registers.count(0)
        if V > 0:
            return m * math.log(m / V)
    elif raw_estimate > (2**32) / 30:
        return -(2**32) * math.log(1 - raw_estimate / (2**32))
    return raw_estimate

def estimate_cardinality(registers):
    m = len(registers)
    alpha_m = 0.7213 / (1 + 1.079 / m)
    raw_estimate = alpha_m * m**2 * harmonic_mean(registers)
    return int(round(bias_correction(registers, raw_estimate, m),0))

# -----------------------------------
# FUNÇÃO PARA OBTER NÚMERO DE EVENTOS
# -----------------------------------

def eventos():
    con = duckdb.connect("github_archive.duckdb")
    n_users_exato_query = con.execute("""
        SELECT COUNT(DISTINCT actor_id)
        FROM github_events
        WHERE actor_id IS NOT NULL""").fetchone()[0]
    con.close()
    return n_users_exato_query

# -----------------------------------
# APLICAR HYPERLOGLOG
# -----------------------------------

m = 1024  # número de registos, podes alterar
registers = [0] * m

for user in df['actor_login']:
    process_element(user, registers, m)

estimate = estimate_cardinality(registers)
n_real_dataframe = df["actor_login"].nunique() #eventos()  # número real de eventos
n_real_query = eventos()  # número real de eventos
erro_percentual = abs(100 * (estimate / n_real_query - 1))
erro_abs = abs(estimate /n_real_dataframe - 1) * 100

print(f"Estimativa de utilizadores únicos (HLL): {estimate:,}")
print(f"Valor Real de utilizadores únicos: {n_real_dataframe:,}\nRespectivo erro percentual: {erro_abs:.2f}%")
print(f"Valor Real de utilizadores únicos (Query): {n_real_query:,}\nRespectivo erro percentual: {erro_percentual:.2f}%")


Estimativa de utilizadores únicos (HLL): 1,511,083
Valor Real de utilizadores únicos: 1,123,357
Respectivo erro percentual: 34.51%
Valor Real de utilizadores únicos (Query): 1,122,592
Respectivo erro percentual: 34.61%


In [68]:
# -----------------------------------
# FUNÇÕES HLL
# -----------------------------------

def hash_function(value):
    hash_value = hashlib.sha256(str(value).encode('utf8')).hexdigest()
    return int(hash_value, 16)

def leftmost_1_bit_position(hash_value):
    bin_hash = bin(hash_value)[2:]
    for i, bit in enumerate(bin_hash):
        if bit == '1':
            return i + 1
    return len(bin_hash) + 1

def process_element(element, registers, m):
    hash_value = hash_function(element)
    p = int(math.log2(m))
    register_index = hash_value & (m - 1)
    remaining_hash = hash_value >> p
    position = leftmost_1_bit_position(remaining_hash)
    registers[register_index] = max(registers[register_index], position)

def harmonic_mean(registers):
    sum_of_inverses = sum([2**-reg for reg in registers])
    return len(registers) / sum_of_inverses

def bias_correction(registers, raw_estimate, m):
    if raw_estimate <= 2.5 * m:
        V = registers.count(0)
        if V > 0:
            return m * math.log(m / V)
    elif raw_estimate > (2**32) / 30:
        return -(2**32) * math.log(1 - raw_estimate / (2**32))
    return raw_estimate

def estimate_cardinality(registers, a, b):
    m = len(registers)
    alpha_m = 0.7213 / (a + b / m)
    raw_estimate = alpha_m * m**2 * harmonic_mean(registers)
    return int(round(bias_correction(registers, raw_estimate, m),0))

def hyperloglog_estimate(dataset, m, n, a, b):
    registers = [0] * m
    for element in dataset:
        process_element(element, registers, m)
    estimate =  int(round(estimate_cardinality(registers, a, b),0))
    error = abs(100*(estimate/n - 1))
    return estimate, error, registers

# -----------------------------------
# FUNÇÃO PARA OBTER NÚMERO DE EVENTOS
# -----------------------------------

def eventos():
    con = duckdb.connect("github_archive.duckdb")
    n_users_exato_query = con.execute("""
        SELECT COUNT(DISTINCT actor_id)
        FROM github_events
        WHERE actor_id IS NOT NULL""").fetchone()[0]
    con.close()
    return n_users_exato_query



# -----------------------------------
# APLICAR HYPERLOGLOG
# -----------------------------------
"""dataset = df['actor_login'].tolist()
n = df['actor_login'].nunique()
pL = []; errorL = []
min_p = None; min_estimate = None; min_error = 9999999
for p in range(10,12):
    m = 2**p     #Número de buckets
    estimate = None; error = 9999999
    for n_a in range(1,11): 
        for n_b in range(1,11):
            a = 0.7213 * n_a/5; b = 1.079 * n_b/5
            estimateT, errorT, registers = hyperloglog_estimate(dataset, m, n, a, b)
            if (error>errorT):
                error = errorT
                estimate = estimateT
    pL.append(p); errorL.append(error)
    if (min_error>error):
        min_p = p
        min_error = error
        min_estimate = estimate
        
print(f"#dataset={len(dataset)} real_difNums={n} estimate_difNums={min_estimate} min(p)={min_p} min(error)={min_error:.2f}%")
sz1 = sys.getsizeof(registers)
sz2 = sys.getsizeof(dataset)
print(f"#registers={2**min_p} size(registers)={sz1} size(dataset)={sz2} Perc={100*sz1/sz2:.2f}%")
plt.plot(pL,errorL)

"""

m = 2**11  # número de registos, podes alterar
registers = [0] * m

for user in df['actor_login']:
    process_element(user, registers, m)

estimate = estimate_cardinality(registers, 1.1, 1.079)
n_real_dataframe = df["actor_login"].nunique() #eventos()  # número real de eventos
n_real_query = eventos()  # número real de eventos
erro_percentual = abs(100 * (estimate / n_real_query - 1))
erro_abs = abs(estimate /n_real_dataframe - 1) * 100

print(f"Estimativa de utilizadores únicos (HLL): {estimate:,}")
print(f"Valor Real de utilizadores únicos: {n_real_dataframe:,}\nRespectivo erro percentual: {erro_abs:.2f}%")
print(f"Valor Real de utilizadores únicos (Query): {n_real_query:,}\nRespectivo erro percentual: {erro_percentual:.2f}%")

Estimativa de utilizadores únicos (HLL): 5,498,006
Valor Real de utilizadores únicos: 1,123,357
Respectivo erro percentual: 389.43%
Valor Real de utilizadores únicos (Query): 1,122,592
Respectivo erro percentual: 389.76%


- Ler artigo HLL google HyperLogLog in Practice: Algorithmic Engineering of a
State of The Art Cardinality Estimation Algorithm e 2007
- Ler Chabchoub et al., Sliding HyperLogLog for Port Scan Detection, 2014
- Calcular valor optimo mudando os parametro m, a e b.
- Comparar a memoria consumida, com a query sql
- Comparar o consumo de memoria para o HLL by hand, HLL python, HLL redis

### 7.2 HLL using a dedicated function

In [ ]:
import duckdb
import pandas as pd
import hyperloglog

print(f"Total de eventos lidos: {len(df):,}")

# --------------------------------------------------
# APLICAR HYPERLOGLOG
# --------------------------------------------------
hll = hyperloglog.HyperLogLog(0.01)

for actor in df['actor_id']:
    hll.add(actor)

estimate = len(hll)
print(f"\nEstimativa de utilizadores únicos (HLL): {estimate:,}")

# --------------------------------------------------
# VALOR EXATO PARA COMPARAÇÃO
# --------------------------------------------------
con = duckdb.connect("github_archive.duckdb")
n_users_exato_query = con.execute("""
    SELECT COUNT(DISTINCT actor_id)
    FROM github_events
    WHERE actor_id IS NOT NULL
""").fetchone()[0]
con.close()

n_users_exato_query_len = df["actor_id"].nunique()

print(f"Número exato de utilizadores únicos contados através de Query: {n_users_exato_query:,}")
print(f"Número exato de utilizadores únicos contados através de DataFrame: {n_users_exato_query_len:,}")

# --------------------------------------------------
# ERRO PERCENTUAL
# --------------------------------------------------
erro_percentual_query = abs(100 * (estimate / n_users_exato_query - 1))
erro_percentual_df = abs(100 * (estimate / n_users_exato_query_len - 1))

print(f"Erro percentual do HLL vs valor exato (Query): {erro_percentual_query:.2f}%")
print(f"Erro percentual do HLL vs valor exato (DataFrame): {erro_percentual_df:.2f}%")


Total de eventos lidos: 10,174,100

Estimativa de utilizadores únicos (HLL): 1,139,943
Número exato de utilizadores únicos contados através de Query: 1,122,592
Número exato de utilizadores únicos contados através de DataFrame: 1,122,592
Erro percentual do HLL vs valor exato (Query): 1.55%
Erro percentual do HLL vs valor exato (DataFrame): 1.55%


### 7.3 Redis HLL Container
- Apply redis HLL
- Look up to other HLL
- Apply Bloom Filters to verify if a actor_id is in a set

In [ ]:

# Conexão ao Redis (assumindo que corre em localhost:6379)
r = redis.Redis(host='localhost', port=6379, db=0)
# Chave no Redis
HLL_KEY = "dataset:hll"

# Limpar antes de começar (caso já exista)
r.delete(HLL_KEY)

# Adicionar cada valor ao HyperLogLog
for value in df['actor_id']:
    r.pfadd(HLL_KEY, value)

redis_estimate = r.pfcount(HLL_KEY)
print(f"Real distinto: {n_real}']}, Redis estima: {redis_estimate}")
error = abs(redis_estimate / n_real -1) * 100
print(f"Erro do Redis: {error:.2f}%")
print(f"Python estimate = {estimate:.2f}")
print(f"Redis HLL estimate = {redis_estimate}")
print(f"Erro customizado = {erro_percentual_df:.2f}%")
print(f"Erro Redis = {abs(redis_estimate - n_real) / n_real * 100:.2f}%")

### 7.4 Results Visualization
In this section you can analyse the results in a ilustrative way...

<br><br>

## 8. Analysis of Example 1 [3,0 valor]

<br><br>

## 9. Analysis of Example 2 [3,0 valor]

<br><br>
## 10. Pros and cons of the approach [2,0 valor]

<br><br>
## 11. Future improvements [2,0 valor]

<br>
<div style="text-align: center;">
    <br><br>
    <p style="font-size: 40px;">References [1,0 valor]</p>
</div>
<br>
